In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
dataset = pd.read_csv('Emotify/data.csv')
dataset.head()

,track id,genre,amazement,solemnity,tenderness,nostalgia,calmness,power,joyful_activation,tension,sadness,mood,liked,disliked,age,gender,mother tongue
0,1,classical,0,1,0,0,0,0,1,1,0,3,1,0,21,1,English
1,1,classical,0,0,0,1,0,0,0,0,0,3,0,1,41,1,Dutch
2,1,classical,0,0,0,1,0,0,0,0,1,3,0,0,24,1,English
3,1,classical,0,0,0,0,1,0,0,0,0,3,0,0,32,0,Spanish
4,1,classical,0,0,0,1,1,0,0,0,0,4,0,1,21,0,English


In [3]:
dataset.columns

Index(['track id', ' genre', ' amazement', ' solemnity', ' tenderness',
       ' nostalgia', ' calmness', ' power', ' joyful_activation', ' tension',
       ' sadness', ' mood', ' liked', ' disliked', ' age', ' gender',
       ' mother tongue'],
      dtype='object')

In [4]:
dataset.columns = dataset.columns.str.strip()

In [5]:
gems_annotations = ['amazement', 'solemnity', 'tenderness', 
                    'nostalgia', 'calmness', 'power', 
                    'joyful_activation', 'tension', 'sadness']
print(dataset['genre'].unique())
print(dataset[gems_annotations])

['classical' 'rock' 'electronic' 'pop']
      amazement  solemnity  tenderness  nostalgia  calmness  power  \
0             0          1           0          0         0      0   
1             0          0           0          1         0      0   
2             0          0           0          1         0      0   
3             0          0           0          0         1      0   
4             0          0           0          1         1      0   
...         ...        ...         ...        ...       ...    ...   
8402          1          1           0          0         0      0   
8403          0          0           0          1         0      0   
8404          0          0           0          0         0      0   
8405          1          0           0          0         0      0   
8406          1          0           0          0         0      0   

      joyful_activation  tension  sadness  
0                     1        1        0  
1                     0        

In [6]:
dataset[gems_annotations].dtypes

amazement            int64
solemnity            int64
tenderness           int64
nostalgia            int64
calmness             int64
power                int64
joyful_activation    int64
tension              int64
sadness              int64
dtype: object

In [7]:
# Group by track ID and genre, summing up the GEMS values
data = dataset.groupby(["track id", "genre"])[gems_annotations].sum().reset_index()

# Compute the count per group
count = dataset.groupby(["track id", "genre"]).size().reset_index(name="count")

# Merge count into the summed data
data = data.merge(count, on=["track id", "genre"], how="left")

data.to_csv("Emotify/summed_emotify.csv", index=False)  # Saves without the index column

In [8]:
def softmax(x):
    exp_x = np.exp(x - np.max(x))  # Subtract max for numerical stability
    return exp_x / np.sum(exp_x)
data.head()

,track id,genre,amazement,solemnity,tenderness,nostalgia,calmness,power,joyful_activation,tension,sadness,count
0,1,classical,7,16,10,14,30,1,4,3,15,48
1,2,classical,5,8,20,16,35,2,1,1,5,47
2,3,classical,8,16,6,8,3,13,19,11,7,46
3,4,classical,5,6,22,19,32,1,0,1,6,42
4,5,classical,2,13,14,23,14,2,3,8,14,44


In [36]:
for index, row in data.iterrows():
    print(f"Row {index}: {row['track id']} - {row['genre']} - {row[gems_annotations].values}")
    # Using np.float64 ensures higher numerical stability in the softmax function, especially when dealing with exponentiation
    data_row = np.array(row[gems_annotations].values, dtype=np.float64) / row["count"]
    softmax_values = softmax(data_row)
    print("Data Divided", data_row)
    threshold = int(np.ceil((np.std(data_row) * 10)))
    print("Standard Deviation", threshold)
    # print("Data Softmax", softmax_values)
    
    # Find the threshold value (the Nth largest value)
    top_nth_value = np.partition(data_row, -threshold)[-threshold]  # Gets the Nth largest value

    # Create a binary mask: 1 for values ≥ top_nth_value, 0 otherwise
    binary_arr = (data_row >= top_nth_value).astype(int)

    print(binary_arr)  # Example output: [0 0 0 1 1 1]
    
    data.loc[index, gems_annotations] = binary_arr
    
    print("-" * 100)
    # for col, value in row[gems_annotations].items():
    #     print(data.at[index, col])

Row 0: 1 - classical - [0 1 0 0 1 0 0 0 0]
Data Divided [0.         0.02083333 0.         0.         0.02083333 0.
 0.         0.         0.        ]
Standard Deviation 1
[0 1 0 0 1 0 0 0 0]
----------------------------------------------------------------------------------------------------
Row 1: 2 - classical - [0 0 1 1 1 0 0 0 0]
Data Divided [0.        0.        0.0212766 0.0212766 0.0212766 0.        0.
 0.        0.       ]
Standard Deviation 1
[0 0 1 1 1 0 0 0 0]
----------------------------------------------------------------------------------------------------
Row 2: 3 - classical - [0 1 0 0 0 0 1 0 0]
Data Divided [0.         0.02173913 0.         0.         0.         0.
 0.02173913 0.         0.        ]
Standard Deviation 1
[0 1 0 0 0 0 1 0 0]
----------------------------------------------------------------------------------------------------
Row 3: 4 - classical - [0 0 1 1 1 0 0 0 0]
Data Divided [0.         0.         0.02380952 0.02380952 0.02380952 0.
 0.         0.   

In [37]:
data.to_csv("Emotify/multi_hot_encoded_emotify.csv", index=False)  # Saves without the index column